In [3]:
from bs4 import BeautifulSoup, Tag, NavigableString
import requests
import time
import json

In [143]:
#with open('animeindex.csv', 'r') as f:
#  reader = csv.reader(f)
#  animelist = list(reader)


In [4]:
def malkey(x):
    """returns an array of primary key[s] from url string[s]"""

    if isinstance(x, str):
        return [int(s) for s in x.split('/') if s.isdigit()]
    
    elif isinstance(x, list):
        if len(x) > 1:
            return [malkey(y) for y in x]  
        else:
            return malkey(x[0])
    
    elif isinstance(x, Tag):
        if len(x) == 1:
            if isinstance(x.contents[0], Tag):
                return malkey(x.contents[0]['href'])
            elif isinstance(x.contents[0], NavigableString):
                return malkey(x['href'])
            else:
                raise Exception('Detected neither Tag nor NavigableString')
        else:
            return [malkey(y)[0] for y in x if isinstance(y, Tag) and len(y) > 0]
                
    else:
        raise Exception('Detected neither str nor list of str')

# Sitemap Update

In [5]:
with open('urllist.txt', 'r', encoding='utf8') as filehandle:  
    animelist = json.load(filehandle)

In [9]:
'https://myanimelist.net/anime/39595/Megazone_23_XI'

True

In [6]:
lastmaxurl = animelist[-1]
lastmaxindex = malkey(lastmaxurl)[0]
lastmaxindex

40305

In [7]:
lastmaxurl

'https://myanimelist.net/anime/40305/Subarashiki_Kokka_no_Kizukikata'

In [8]:
len(animelist)

15955

In [9]:
# empty list for updating
updatelist = []

In [140]:
# lastmaxurl = 'https://myanimelist.net/anime/39565/Boku_no_Hero_Academia_the_Movie_2'
# lastmaxindex = 39565

## Get first page

In [10]:
minlooptime = 2

In [11]:
page = requests.get('https://myanimelist.net/anime.php?o=9&c%5B0%5D=a&c%5B1%5D=d&cv=2&w=1')

In [12]:
firstpage = BeautifulSoup(page.content, 'html.parser')

In [13]:
table = firstpage.select('.borderClass')
urllist = [x.contents[1].contents[1]['href'] for x in table if len(x) == 3]

In [14]:
malkey(urllist[49])[0]

40239

In [15]:
if malkey(urllist[49])[0] > lastmaxindex:
    increment = 0

    while malkey(urllist[49])[0] > lastmaxindex:
        timestart = time.time()
        updatelist.extend(urllist)
        
        increment += 50
        page = requests.get('https://myanimelist.net/anime.php?o=9&c%5B0%5D=a&c%5B1%5D=d&cv=2&w=1&show=' + str(increment))
        print(page)
        nextpage = BeautifulSoup(page.content, 'html.parser')
    
        table = nextpage.select('.borderClass')
        urllist = [x.contents[1].contents[1]['href'] for x in table if len(x) == 3]
        print(malkey(urllist[49])[0])

        if time.time() - timestart < minlooptime:
            time.sleep(minlooptime - (time.time() - timestart))

print('Reached converged page')

Reached converged page


In [16]:
updatelist

[]

In [17]:
urllist

['https://myanimelist.net/anime/40351/Pokemon_Shin_Series',
 'https://myanimelist.net/anime/40344/Baumkuchen_End',
 'https://myanimelist.net/anime/40342/Taka_no_Tsume-dan_x_Uchicomi',
 'https://myanimelist.net/anime/40338/Iro_wo_Kuu_Akuma',
 'https://myanimelist.net/anime/40337/Aibeya_The_Animation',
 'https://myanimelist.net/anime/40336/Love_x_Holic__Miwaku_no_Otome_to_Hakudaku_Kankei_-_The_Animation',
 'https://myanimelist.net/anime/40335/Kimi_dake_ni_Motetainda',
 'https://myanimelist.net/anime/40334/Code_Geass__Hangyaku_no_Lelouch_Picture_Drama_-_Kiseki_no_Anniversary',
 'https://myanimelist.net/anime/40333/Uzumaki',
 'https://myanimelist.net/anime/40332/Kyochuu_Rettou_Movie',
 'https://myanimelist.net/anime/40331/Ongaku',
 'https://myanimelist.net/anime/40329/Crayon_Shin-chan_Movie_28__Gekitotsu_Rakugaki_Kingdom_to_Hobo_Yonin_no_Yuusha',
 'https://myanimelist.net/anime/40327/Mewkledreamy',
 'https://myanimelist.net/anime/40323/Gintama__Monster_Strike-hen',
 'https://myanimelist.ne

In [18]:
len(urllist)

50

In [19]:
lastpage = urllist[0:urllist.index(lastmaxurl)]
lastpage

['https://myanimelist.net/anime/40351/Pokemon_Shin_Series',
 'https://myanimelist.net/anime/40344/Baumkuchen_End',
 'https://myanimelist.net/anime/40342/Taka_no_Tsume-dan_x_Uchicomi',
 'https://myanimelist.net/anime/40338/Iro_wo_Kuu_Akuma',
 'https://myanimelist.net/anime/40337/Aibeya_The_Animation',
 'https://myanimelist.net/anime/40336/Love_x_Holic__Miwaku_no_Otome_to_Hakudaku_Kankei_-_The_Animation',
 'https://myanimelist.net/anime/40335/Kimi_dake_ni_Motetainda',
 'https://myanimelist.net/anime/40334/Code_Geass__Hangyaku_no_Lelouch_Picture_Drama_-_Kiseki_no_Anniversary',
 'https://myanimelist.net/anime/40333/Uzumaki',
 'https://myanimelist.net/anime/40332/Kyochuu_Rettou_Movie',
 'https://myanimelist.net/anime/40331/Ongaku',
 'https://myanimelist.net/anime/40329/Crayon_Shin-chan_Movie_28__Gekitotsu_Rakugaki_Kingdom_to_Hobo_Yonin_no_Yuusha',
 'https://myanimelist.net/anime/40327/Mewkledreamy',
 'https://myanimelist.net/anime/40323/Gintama__Monster_Strike-hen',
 'https://myanimelist.ne

In [20]:
len(lastpage)

21

In [21]:
updatelist.extend(lastpage)

In [22]:
updatelist

['https://myanimelist.net/anime/40351/Pokemon_Shin_Series',
 'https://myanimelist.net/anime/40344/Baumkuchen_End',
 'https://myanimelist.net/anime/40342/Taka_no_Tsume-dan_x_Uchicomi',
 'https://myanimelist.net/anime/40338/Iro_wo_Kuu_Akuma',
 'https://myanimelist.net/anime/40337/Aibeya_The_Animation',
 'https://myanimelist.net/anime/40336/Love_x_Holic__Miwaku_no_Otome_to_Hakudaku_Kankei_-_The_Animation',
 'https://myanimelist.net/anime/40335/Kimi_dake_ni_Motetainda',
 'https://myanimelist.net/anime/40334/Code_Geass__Hangyaku_no_Lelouch_Picture_Drama_-_Kiseki_no_Anniversary',
 'https://myanimelist.net/anime/40333/Uzumaki',
 'https://myanimelist.net/anime/40332/Kyochuu_Rettou_Movie',
 'https://myanimelist.net/anime/40331/Ongaku',
 'https://myanimelist.net/anime/40329/Crayon_Shin-chan_Movie_28__Gekitotsu_Rakugaki_Kingdom_to_Hobo_Yonin_no_Yuusha',
 'https://myanimelist.net/anime/40327/Mewkledreamy',
 'https://myanimelist.net/anime/40323/Gintama__Monster_Strike-hen',
 'https://myanimelist.ne

In [23]:
updatelist.reverse()

In [24]:
updatelist

['https://myanimelist.net/anime/40312/Southpaw',
 'https://myanimelist.net/anime/40313/Promare__Lio-hen',
 'https://myanimelist.net/anime/40314/Sekaiichi_Hatsukoi__Propose-hen',
 'https://myanimelist.net/anime/40315/Code_Geass__Fukkatsu_no_Lelouch_-_Shinkai_no_Kakera',
 'https://myanimelist.net/anime/40316/Phantomi__Mini_Anime',
 'https://myanimelist.net/anime/40320/Tenka_Hyakken__Meiji-kan_e_Youkoso',
 'https://myanimelist.net/anime/40321/Ookami_to_Koushinryou_VR_2',
 'https://myanimelist.net/anime/40323/Gintama__Monster_Strike-hen',
 'https://myanimelist.net/anime/40327/Mewkledreamy',
 'https://myanimelist.net/anime/40329/Crayon_Shin-chan_Movie_28__Gekitotsu_Rakugaki_Kingdom_to_Hobo_Yonin_no_Yuusha',
 'https://myanimelist.net/anime/40331/Ongaku',
 'https://myanimelist.net/anime/40332/Kyochuu_Rettou_Movie',
 'https://myanimelist.net/anime/40333/Uzumaki',
 'https://myanimelist.net/anime/40334/Code_Geass__Hangyaku_no_Lelouch_Picture_Drama_-_Kiseki_no_Anniversary',
 'https://myanimelist.

In [28]:
animelist.extend(updatelist)

In [29]:
animelist[0:10]

['https://myanimelist.net/anime/1/Cowboy_Bebop',
 'https://myanimelist.net/anime/5/Cowboy_Bebop__Tengoku_no_Tobira',
 'https://myanimelist.net/anime/6/Trigun',
 'https://myanimelist.net/anime/7/Witch_Hunter_Robin',
 'https://myanimelist.net/anime/8/Bouken_Ou_Beet',
 'https://myanimelist.net/anime/15/Eyeshield_21',
 'https://myanimelist.net/anime/16/Hachimitsu_to_Clover',
 'https://myanimelist.net/anime/17/Hungry_Heart__Wild_Striker',
 'https://myanimelist.net/anime/18/Initial_D_Fourth_Stage',
 'https://myanimelist.net/anime/19/Monster']

In [30]:
animelist[-10:]

['https://myanimelist.net/anime/40332/Kyochuu_Rettou_Movie',
 'https://myanimelist.net/anime/40333/Uzumaki',
 'https://myanimelist.net/anime/40334/Code_Geass__Hangyaku_no_Lelouch_Picture_Drama_-_Kiseki_no_Anniversary',
 'https://myanimelist.net/anime/40335/Kimi_dake_ni_Motetainda',
 'https://myanimelist.net/anime/40336/Love_x_Holic__Miwaku_no_Otome_to_Hakudaku_Kankei_-_The_Animation',
 'https://myanimelist.net/anime/40337/Aibeya_The_Animation',
 'https://myanimelist.net/anime/40338/Iro_wo_Kuu_Akuma',
 'https://myanimelist.net/anime/40342/Taka_no_Tsume-dan_x_Uchicomi',
 'https://myanimelist.net/anime/40344/Baumkuchen_End',
 'https://myanimelist.net/anime/40351/Pokemon_Shin_Series']

In [165]:
# animelist = [x[0] for x in animelist]

In [49]:
# animelist[0:10]

In [167]:
#with open('pythonanimelist.csv', 'w', encoding='utf-8') as myfile:
#     wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
#     wr.writerow(animelist)

In [31]:
with open('urllist.txt', 'w', encoding='utf8') as filehandle:  
    json.dump(animelist, filehandle)

# MAL Dataframe partial update

## New Sitemap Append

In [32]:
import pandas as pd

In [33]:
# load maldf
maldf = pd.read_json(r'maldf.json', orient='index')
maldf.sort_index(inplace=True)
maldf.head()

,Aired,Broadcast,Duration,English,Episodes,Favorites,Genres,Japanese,Licensors,Members,...,Rating,Score,ScoreExtract,ScoreRaters,Source,Status,Studios,Synonyms,Title,Type
1,"Apr 3, 1998 to Apr 24, 1999",Saturdays at 01:00 (JST),24 min. per ep.,Cowboy Bebop,26,"46,627","Action,Adventure,Comedy,Drama,Sci-Fi,Space",カウボーイビバップ,"Funimation,Bandai Entertainment","864,330",...,R - 17+ (violence & profanity),"8.821(scored by435,335users)1\n indicat...",NaN,NaN,Original,Finished Airing,Sunrise,None,Cowboy Bebop,TV
5,"Sep 1, 2001",None,1 hr. 55 min.,Cowboy Bebop: The Movie,1,849,"Action,Drama,Mystery,Sci-Fi,Space",カウボーイビバップ 天国の扉,Sony Pictures Entertainment,"211,101",...,R - 17+ (violence & profanity),"8.411(scored by126,733users)1\n indicat...",NaN,NaN,Original,Finished Airing,Bones,Cowboy Bebop: Knockin' on Heaven's Door,Cowboy Bebop: Tengoku no Tobira,Movie
6,"Apr 1, 1998 to Sep 30, 1998",Thursdays at 01:15 (JST),24 min. per ep.,Trigun,26,"10,919","Action,Sci-Fi,Adventure,Comedy,Drama,Shounen",トライガン,"Funimation,Geneon Entertainment USA","434,992",...,PG-13 - Teens 13 or older,"8.291(scored by222,984users)1\n indicat...",NaN,NaN,Manga,Finished Airing,Madhouse,None,Trigun,TV
7,"Jul 2, 2002 to Dec 24, 2002",Tuesdays at Unknown,25 min. per ep.,Witch Hunter Robin,26,548,"Action,Magic,Police,Supernatural,Drama,Mystery",Witch Hunter ROBIN,"Funimation,Bandai Entertainment","82,627",...,PG-13 - Teens 13 or older,"7.321(scored by33,573users)1\n indicate...",NaN,NaN,Original,Finished Airing,Sunrise,WHR,Witch Hunter Robin,TV
8,"Sep 30, 2004 to Sep 29, 2005",Thursdays at 18:30 (JST),23 min. per ep.,Beet the Vandel Buster,52,15,"Adventure,Fantasy,Shounen,Supernatural",冒険王ビィト,"None found,add some","12,055",...,PG - Children,"7.031(scored by4,996users)1\n indicates...",NaN,NaN,Manga,Finished Airing,Toei Animation,Adventure King Beet,Bouken Ou Beet,TV


In [34]:
maxdf = maldf.index.max()
maxdf

40305

In [35]:
# remane to urllist to keep Loop code consistent
urllist = [x for x in animelist if malkey(x)[0] > maxdf]

In [36]:
len(urllist)

21

In [37]:
def tagstringextract(x):
    """extract string from NavigableString"""
    if isinstance(x, NavigableString):
        return x.lstrip().rstrip()
    elif isinstance(x, Tag):
        return x.get_text().lstrip().rstrip()
    elif isinstance(x, list):
        return ''.join([tagstringextract(a) for a in x])
    else:
        raise Exception('Detected neither Tag nor NavigableString')
        
def chunks(l, n):
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i:i + n]

In [38]:
mallist = []
failedlist = []
sequelkeys = []
maxindex = len(urllist)
minlooptime = 2
failedcount = 0

In [40]:
for idx, url in enumerate(urllist):
    # start timer
    timestart = time.time()
    
    try:
        page = requests.get(url,timeout=5)
        page.raise_for_status()
    except requests.exceptions.RequestException as e:
        failedcount += 1 
        print(e)
        print('Current Failed Count: ' + str(failedcount))
        failedlist.append(url)
        
        if time.time() - timestart < minlooptime:
            time.sleep(minlooptime - (time.time() - timestart))
        continue

    
    # get primary key, start new row
    animekey = malkey(url)[0]
    newrow = {'key': animekey}
    
    # create soup
    anime = BeautifulSoup(page.content, 'html.parser')
    
    # get title
    title = {'Title': anime.select('.h1 span')[0].string}
    newrow.update(title)
    
    # get sidebar information
    sidebar = [a for a in 
               [[x for x in entry if x != '\n'] for entry in anime.select('.js-scrollfix-bottom div')]
               if len(a) > 1 and a[0].name == 'span']
    sidebardict = dict([entry[0].get_text(), tagstringextract(entry[1:])] for entry in sidebar)
    newrow.update(sidebardict)
    
    # add to global list
    mallist.append(newrow)
    
    # related
    relatedanime = anime.select('.anime_detail_related_anime .borderClass')
    relatedlist = [[animekey, alpha[0].text, y] for alpha in chunks(relatedanime, 2) for y in malkey(alpha[1])]
    
    sequelkeys += relatedlist
    
    # pause if too fast
    if time.time() - timestart < minlooptime:
        time.sleep(minlooptime - (time.time() - timestart))
    print('Processed anime [key = ' + str(animekey) + '], ' 
          + '%d' % (idx+1) + '/' + str(maxindex) + ', ' + '%.2f' % (100*(idx+1)/maxindex) + '%: in ' 
          + '%.4f' % (time.time() - timestart) + ' seconds.')


Processed anime [key = 40312], 1/21, 4.76%: in 2.0012 seconds.
Processed anime [key = 40313], 2/21, 9.52%: in 2.0014 seconds.
Processed anime [key = 40314], 3/21, 14.29%: in 2.0015 seconds.
Processed anime [key = 40315], 4/21, 19.05%: in 2.0016 seconds.
Processed anime [key = 40316], 5/21, 23.81%: in 2.0009 seconds.
Processed anime [key = 40320], 6/21, 28.57%: in 2.0006 seconds.
Processed anime [key = 40321], 7/21, 33.33%: in 2.0009 seconds.
Processed anime [key = 40323], 8/21, 38.10%: in 2.0016 seconds.
Processed anime [key = 40327], 9/21, 42.86%: in 2.0011 seconds.
Processed anime [key = 40329], 10/21, 47.62%: in 2.0011 seconds.
Processed anime [key = 40331], 11/21, 52.38%: in 2.0007 seconds.
Processed anime [key = 40332], 12/21, 57.14%: in 2.0009 seconds.
Processed anime [key = 40333], 13/21, 61.90%: in 2.0014 seconds.
Processed anime [key = 40334], 14/21, 66.67%: in 2.0009 seconds.
Processed anime [key = 40335], 15/21, 71.43%: in 2.0016 seconds.
Processed anime [key = 40336], 16/21

In [41]:
len(mallist)

21

In [42]:
maldfnew = pd.DataFrame(mallist)

In [43]:
maldfnew = maldfnew.set_index('key')

In [44]:
maldfnew.columns = maldfnew.columns.str.replace(':', '')

In [45]:
maldfnew['Episodes'] = maldfnew['Episodes'].replace({',',''})
maldfnew['Episodes'] = maldfnew['Episodes'].replace('Unknown', '0')
maldfnew.Episodes = maldfnew.Episodes.astype('int32')

In [46]:
maldfnew['Members'] = maldfnew['Members'].str.replace(',','').astype('int64')
maldfnew['Favorites'] = maldfnew['Favorites'].str.replace(',','').astype('int64')

In [47]:
maldfnew['ScoreExtract'] = maldfnew.Score.str.extract('(\d+.\d+)').astype('float64')

In [48]:
maldfnew['ScoreRaters'] = maldfnew.Score.str.extract('(?<=by)(.*)(?=users)')
maldfnew['ScoreRaters'] = maldfnew['ScoreRaters'].str.replace(',', '').astype('int64')

In [49]:
maldfupdated = pd.concat([maldf, maldfnew], sort=False)

In [50]:
maldfupdated.tail()

,Aired,Broadcast,Duration,English,Episodes,Favorites,Genres,Japanese,Licensors,Members,...,Rating,Score,ScoreExtract,ScoreRaters,Source,Status,Studios,Synonyms,Title,Type
40337,"Nov 29, 2019",NaN,30 min.,NaN,1,0,Hentai,アイベヤ THE ANIMATION,"None found,add some",195,...,Rx - Hentai,N/A1(scored by0users)1\n This score is ...,NaN,0.0,Visual novel,Not yet aired,"None found,add some",NaN,Aibeya The Animation,OVA
40338,"Aug 24, 2015",NaN,1 min.,The Color Eater,1,0,"Dementia,Music",色を喰うアクマ,"None found,add some",86,...,G - All Ages,N/A1(scored by0users)1\n This score is ...,NaN,0.0,Original,Finished Airing,"None found,add some",NaN,Iro wo Kuu Akuma,ONA
40342,"Aug 19, 2019 to Aug 29, 2019",NaN,1 min. per ep.,NaN,5,0,"Comedy,Parody,Slice of Life",鷹の爪団×ウチコミ！,"None found,add some",33,...,G - All Ages,4.331(scored by3users)1\n This score is...,4.331,3.0,Original,Finished Airing,"None found,add some","Eagle Talon, Himitsukessha Taka no Tsume, The ...",Taka no Tsume-dan x Uchicomi!,ONA
40344,"Aug 29, 2019",NaN,3 min.,Baumkuchen End,1,0,Music,Baumkuchen End,"None found,add some",22,...,G - All Ages,6.451(scored by11users)1\n This score i...,6.451,11.0,Original,Finished Airing,"None found,add some",Eve,Baumkuchen End,Music
40351,Not available,Unknown,Unknown,Pokémon,0,3,"Action,Adventure,Comedy,Kids,Fantasy",ポケットモンスター,"None found,add some",1931,...,PG - Children,N/A1(scored by0users)1\n This score is ...,NaN,0.0,Game,Not yet aired,"None found,add some",Pocket Monsters,Pokemon (Shin Series),TV


In [52]:
#maldfupdated.to_json(r'maldf.json', orient='index')

### Sequelkeys update

In [53]:
with open(r'sequelkeys.txt', 'r', encoding='utf8') as filehandle:  
    sequelkeyold = json.load(filehandle)

In [54]:
len(sequelkeyold)

22722

In [55]:
sequelkeyold[0:10]

[[1, 'Adaptation', 173],
 [1, 'Adaptation', 174],
 [1, 'Side story', 5],
 [1, 'Side story', 17205],
 [1, 'Summary', 4037],
 [5, 'Parent story', 1],
 [6, 'Adaptation', 703],
 [6, 'Side story', 4106],
 [8, 'Adaptation', 1348],
 [8, 'Sequel', 1123]]

In [56]:
# trim trailing ':' in sequel type
for i in sequelkeys:
    i[1] = i[1][:-1]

sequelkeys[0:10]

[[40313, 'Sequel', 35848],
 [40313, 'Other', 39991],
 [40314, 'Adaptation', 10573],
 [40314, 'Prequel', 11123],
 [40315, 'Parent story', 34437],
 [40321, 'Prequel', 38026],
 [40323, 'Character', 918],
 [40323, 'Character', 30757],
 [40327, 'Adaptation', 121506],
 [40329, 'Parent story', 966]]

In [57]:
len(sequelkeys)

20

In [58]:
sequelkeys[0:10]

[[40313, 'Sequel', 35848],
 [40313, 'Other', 39991],
 [40314, 'Adaptation', 10573],
 [40314, 'Prequel', 11123],
 [40315, 'Parent story', 34437],
 [40321, 'Prequel', 38026],
 [40323, 'Character', 918],
 [40323, 'Character', 30757],
 [40327, 'Adaptation', 121506],
 [40329, 'Parent story', 966]]

In [59]:
sequelkeysupdated = sequelkeyold + sequelkeys

In [60]:
len(sequelkeysupdated)

22742

In [61]:
#with open('sequelkeys.txt', 'w', encoding='utf8') as filehandle:  
#    json.dump(sequelkeysupdated, filehandle)

# Partial Anime Update

In [ ]:
from datetime import datetime
from dateutil.relativedelta import relativedelta

In [ ]:
def seasonurl(now, offset = 0):
    date = now + relativedelta(months=offset)
    if date.month <= 3:
        return str(date.year)+'/winter'
    if date.month <= 6:
        return str(date.year)+'/spring'
    if date.month <= 9:
        return str(date.year)+'/summer'
    if date.month <= 12:
        return str(date.year)+'/fall'

In [ ]:
now = datetime.now()
nowyear = now.year
nowmonth = now.month

### Get Current Season map

In [ ]:
page = requests.get('https://myanimelist.net/anime/season/' + seasonurl(now))

In [ ]:
season = BeautifulSoup(page.content, 'html.parser')

In [ ]:
seasonanime = season.select('.link-title')

## Full Sitemap Refresh

In [55]:
lastmaxurl = 'https://myanimelist.net/anime/1/Cowboy_Bebop'
lastmaxindex = 1

#### First Page

In [4]:
page = requests.get('https://myanimelist.net/anime.php?o=9&c%5B0%5D=a&c%5B1%5D=d&cv=2&w=1')
firstpage = BeautifulSoup(page.content, 'html.parser')

table = firstpage.select('.borderClass')
urllist = [x.contents[1].contents[1]['href'] for x in table if len(x) == 3]

In [5]:
page.status_code

200

In [10]:
increment = 0
updatelist = []

In [7]:
increment

0

In [65]:
# if failed and restarting
increment -= 50

In [11]:

# while increment < 1000:
while malkey(urllist[-1])[0] > lastmaxindex:

    looptime = time.time()
    updatelist.extend(urllist)
    
    increment += 50
    page = requests.get('https://myanimelist.net/anime.php?o=9&c%5B0%5D=a&c%5B1%5D=d&cv=2&w=1&show=' + str(increment))
    while page.status_code != 200:
        print('Error: ' + str(page.status_code) + '. Last scraped: ' + str(malkey(urllist[-1])[0]))
        print('Pausing')
        time.sleep(60)
        page = requests.get('https://myanimelist.net/anime.php?o=9&c%5B0%5D=a&c%5B1%5D=d&cv=2&w=1&show=' + str(increment))
        
    nextpage = BeautifulSoup(page.content, 'html.parser')
    
    table = nextpage.select('.borderClass')
    urllist = [x.contents[1].contents[1]['href'] for x in table if len(x) == 3]
    print(malkey(urllist[49])[0])
    
    
    if time.time() - looptime < 2.5:
        time.sleep(2.5 - (time.time() - looptime))
    print(time.time() - looptime)

print('Reached converged page')
updatelist.extend(urllist)


39607
2.5003068447113037
39513
2.500631332397461
39413
2.5008692741394043
39326
2.5009701251983643
39252
2.50034236907959
39189
2.501122236251831
39135
2.501394748687744
39075
2.5008275508880615
38993
2.501014232635498
38915
2.5010409355163574
38854
2.5007433891296387
38783
2.501014232635498
38714
2.5011186599731445
38628
2.5015578269958496
38536
2.500944137573242
38470
2.5005855560302734
38378
2.5005242824554443
38301
2.50116229057312
38177
2.50065016746521
38099
2.5011050701141357
38044
2.50089168548584
37992
2.500532388687134
37933
2.500636100769043
37859
2.5013556480407715
37762
2.500643491744995
37682
4.668808937072754
37601
2.5011656284332275
37535
2.501201629638672
37461
2.5009398460388184
37388
2.5007994174957275
37303
2.500385046005249
37215
2.5004467964172363
37144
2.500699758529663
37058
2.5013608932495117
36968
2.5002100467681885
36881
2.5015506744384766
36804
2.5012080669403076
36735
2.5015146732330322
36661
2.500833749771118
36593
2.500927686691284
36521
2.501228570938110

IndexError: list index out of range

In [15]:
updatelist.extend(urllist)

In [16]:
updatelist.reverse()

In [21]:
updatelist[0]

'https://myanimelist.net/anime/1/Cowboy_Bebop'

In [20]:
len(updatelist)

15787

In [56]:
page = requests.get('https://myanimelist.net/anime.php?o=9&c%5B0%5D=a&c%5B1%5D=d&cv=2&w=1&show=' + str(increment))
nextpage = BeautifulSoup(page.content, 'html.parser')
    
table = nextpage.select('.borderClass')
urllist = [x.contents[1].contents[1]['href'] for x in table if len(x) == 3]

In [57]:
urllist

['https://myanimelist.net/anime/29992/Silent_Chaser_Kagami',
 'https://myanimelist.net/anime/29989/Kaze_no_Kokyuu__Animation_ni_Yoru_Oufuku_Shokan',
 'https://myanimelist.net/anime/29988/Commercial_War',
 'https://myanimelist.net/anime/29987/Flicker_Love_No1',
 'https://myanimelist.net/anime/29986/Women',
 'https://myanimelist.net/anime/29985/Kamen_no_Marionette-tachi',
 'https://myanimelist.net/anime/29984/Crayon_Angel',
 'https://myanimelist.net/anime/29980/Adventures_in_Beauty_Wonderland',
 'https://myanimelist.net/anime/29979/Petting_a_Dog',
 'https://myanimelist.net/anime/29978/001',
 'https://myanimelist.net/anime/29976/Dance_with_Devils',
 'https://myanimelist.net/anime/29975/Gunslinger_Stratos',
 'https://myanimelist.net/anime/29974/Diabolik_Lovers_MoreBlood',
 'https://myanimelist.net/anime/29973/Aya_Hito_Shiki_to_Iu_na_no_Ishi_Hata',
 'https://myanimelist.net/anime/29972/Hakuchuu_Meikyuu',
 'https://myanimelist.net/anime/29971/Landmark',
 'https://myanimelist.net/anime/29970/

In [52]:
urllist_first = updatelist

In [54]:
len(urllist_first)

5600

In [55]:
urllist_first[-1]

'https://myanimelist.net/anime/29918/Go-hiki_no_Kozaru-tachi'

In [28]:
malkey(urllist[-1])[0]

39736

In [34]:
print('Error: ' + str(page.status_code) + '. Last scraped: ' + str(malkey(urllist[-1])[0]))

Error: 429. Last scraped: 39252


In [74]:
len(updatelist)

16287

In [75]:
x = set(updatelist)

In [76]:
len(x)

15787

In [78]:
x

{'https://myanimelist.net/anime/986/Dragon_Ball_Z_Special_1__Tatta_Hitori_no_Saishuu_Kessen',
 'https://myanimelist.net/anime/31815/Kono_Danshi_Mahou_ga_Oshigoto_Desu',
 'https://myanimelist.net/anime/32472/Okoru_yo',
 'https://myanimelist.net/anime/34636/Ballroom_e_Youkoso',
 'https://myanimelist.net/anime/39313/Deep_Blue_Town_e_Oide_yo',
 'https://myanimelist.net/anime/28581/Nintama_Rantarou_no_Jishin_Youjin_Hi_no_Youjin',
 'https://myanimelist.net/anime/807/Tsubasa_Chronicle__Tori_Kago_no_Kuni_no_Himegimi',
 'https://myanimelist.net/anime/28929/Vampire_Holmes',
 'https://myanimelist.net/anime/1010/Ranma_½__Chou_Musabetsu_Kessen_Ranma_Team_vs_Densetsu_no_Houou',
 'https://myanimelist.net/anime/274/Kiddy_Grade',
 'https://myanimelist.net/anime/7155/Beat_Shot',
 'https://myanimelist.net/anime/10824/Ranma_½__DoCo_Music_Video',
 'https://myanimelist.net/anime/6377/Zan_Sayonara_Zetsubou_Sensei',
 'https://myanimelist.net/anime/14647/Tantei_Opera_Milky_Holmes__Alternative',
 'https://myani

In [80]:
with open('pythonanimelistNEW.csv', 'w', encoding='utf-8') as myfile:
     wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
     wr.writerow(updatelist)

In [23]:
with open('pythonanimelist3.csv', 'w', encoding='utf-8') as f:
    writer = csv.writer(f)
    for val in updatelist:
        writer.writerow([val])

In [4]:
with open('urllist2.txt', 'w', encoding='utf8') as filehandle:  
    json.dump(urllist, filehandle)